In [1]:
from itertools import combinations, product
import sys
import argparse
import io

mypath = '/Users/cthoyt/GoogleDrive/Bonn/programming_lab_2/'

# Exercise 1

## Exercise 1A

Write a function called middle that takes a list and returns a new list that contains all but the first and last elements. So middle([1,2,3,4]) should return [2,3]

In [2]:
def middle_naive(l):
    if len(l) <= 2:
        return l
    
    result = []
    for i in range(1, len(l) - 1):
        result.append(l[i])
    return result
    
def middle_slice(l):
    if len(l) <= 2:
        return l
    return l[1:-1]

for middle in (middle_naive, middle_slice):
    print(middle)
    assert [] == middle([])
    assert [1] == middle([1])
    assert [2, 3] == middle([1,2,3,4])

<function middle_naive at 0x10ef5c730>
<function middle_slice at 0x10ec18d90>


## Exercise 1B

Write a function called chop that takes a list, modifies it by removing the first and last elements, and returns None.

In [3]:
def chop_inplace(l):
    l.pop(0)
    l.pop()
    return None

def chop_wrong1(l):
    t = []
    for i in range(1, len(l) - 1):
        t.append(l[i])
    l = t
    return None

def chop_wrong2(l):
    l = l[1:-1]
    return None

for chop in (chop_inplace, ):
    l = [1,2,3,4]
    chop(l)
    assert l == [2,3]
    
for chop in (chop_wrong1, chop_wrong2):
    l = [1,2,3,4]
    chop(l)
    assert l == [1,2,3,4]

# Exercise 2

## Exercise 2A

Write a function that takes a list of numbers and returns the cumulative sum; that is, a new list where the ith element is the sum of the first i + 1 elements from the original list. For example, the cumulative sum of [1, 2, 3] is [1, 3, 6].

In [4]:
def cumsum_naive(l):
    result = []
    for i in range(len(l)):
        el = 0
        for j in range(0, i + 1):
            el += l[j]
        result.append(el)
    return result

def cumsum_naive2(l):
    result = []
    for i in range(len(l)):
        result.append(sum(l[:i + 1]))
    return result

import numpy as np

def cumsum_smart(l):
    return list(np.cumsum(l))

for cumsum in (cumsum_naive, cumsum_naive2, cumsum_smart):
    print(cumsum)
    assert [1,3,6] == cumsum([1,2,3])

<function cumsum_naive at 0x10ef5cea0>
<function cumsum_naive2 at 0x10ef5cd90>
<function cumsum_smart at 0x10ef5cd08>


## Exercise 2B

Write a function called nested_sum that takes a nested list of integers and adds up the elements from all of the nested lists.

In [5]:
def nested_sum_naive1(l):
    s = 0
    for i in range(len(l)):
        for j in range(len(l[i])):
            s += l[i][j]
    return s

def nested_sum_naive2(l):
    s = 0
    for subl in l:
        for el in subl:
            s += el
    return s
    
def nested_sum_smart(l):
    return sum(sum(element) for element in l)

for nested_sum in (nested_sum_naive1, nested_sum_naive2, nested_sum_smart):
    print(nested_sum)
    assert 10 == nested_sum([[1,2], [7]])
    assert 0 == nested_sum([[]])
    assert 10 == nested_sum([[1,2,7]])
    assert 10 == nested_sum([[1,2], [7], []])
    assert 10 == nested_sum([[1,2], [7], [0]])

<function nested_sum_naive1 at 0x10f4f1840>
<function nested_sum_naive2 at 0x10ec0af28>
<function nested_sum_smart at 0x10f65c1e0>


## Exercise 2C

What about a list like [1,[2,3,[4,5]],[[6,7],8,[9,[10]]]] that contains numbers nested in lists of arbitrary depth. Write a function depth_sum that takes a list containing integers nested in lists of arbitrary depth and returns the sum of all the numbers. Hint: Use recursion.

In [6]:
def depth_sum(l):
    if isinstance(l, int):
        return l
    elif isinstance(l, list):
        return sum(depth_sum(el) for el in l)
    else:
        raise Exception("Type Error")

assert 6 == depth_sum(6)
assert 0 == depth_sum([0])
assert 0 == depth_sum([])
assert 0 == depth_sum([[]])
assert 0 == depth_sum([[0]])
assert 0 == depth_sum([[], 0])
assert 55 == depth_sum([1,[2,3,[4,5]],[[6,7],8,[9,[10]]]])
assert 55 == depth_sum([1,[2,3,[4,5]],[[6,7],8,[9,[10, []]]]])

# Exercise 3

Reverse Pairs

## Exercise 3A

Write a function that reads the file words.txt and builds a list with one element per word. Write two versions of this function, one using the append method and the other using the idiom t = t + [x]. Which one takes longer to run? Why?

In [7]:
def read_words_cat(path):
    words = []
    with open(path) as f:
        for line in f:
            words += [line.strip()]
    return words

def read_words_append(path):
    words = []
    with open(path) as f:
        for line in f:
            words.append(line.strip())
    return words
    
cwords = read_words_append(mypath + 'words.txt')
print("{} words loaded".format(len(cwords)))
cwords[:6]

113809 words loaded


['aa', 'aah', 'aahed', 'aahing', 'aahs', 'aal']

In [8]:
%timeit read_words_cat(mypath + 'words.txt')

10 loops, best of 3: 52.5 ms per loop


In [9]:
%timeit read_words_append(mypath + 'words.txt')

10 loops, best of 3: 47.2 ms per loop


### Explanation

The concatenation solution is slower because it needs to build a new list for every element. The append solution addresses this, but using the built-in iterator is the best solution because of speed and elegance.

## Exercise 3B

Two words are a “reverse pair” if each is the reverse of the other. Write a program that finds all the reverse pairs in the word list words.txt. To this end, first read in all the words of words.txt in a (sorted) list. You can test whether a word is already in the list using the in operator. However, you can also use the bisect method of ex. 10.10 of Think Python or use the bisect module, which is part of the Python standard library.

In [10]:
def is_reverse_pair_naive(a, b):
    if len(a) != len(b):
        return False
    for i in range(len(a)):
        if a[i] != b[-(i + 1)]:
            return False
    return True

def is_reverse_pair_easy(a, b):
    if len(a) != len(b):
        return False
    return a == b[::-1]

for is_reverse_pair in (is_reverse_pair_naive, is_reverse_pair_easy):
    print(is_reverse_pair)
    assert is_reverse_pair("", "")
    assert not is_reverse_pair("A", "")
    assert not is_reverse_pair("", "A")
    assert is_reverse_pair("ABA", "ABA")
    assert is_reverse_pair("ABC", "CBA")

<function is_reverse_pair_naive at 0x10f652620>
<function is_reverse_pair_easy at 0x10f652598>


In [13]:
def get_reverse_pairs_naive(words):
    pairs = []
    for i in range(len(words)):
        for j in range(i + 1, len(words)):
            if is_reverse_pair_easy(words[i], words[j]):
                pairs.append((words[i], words[j]))
    return pairs            

def get_reverse_pairs_easy(words):
    return [(a, b) for a, b in combinations(words, 2) if is_reverse_pair_naive(a, b)]

def get_reverse_pairs_sets(words):
    result = []
    for word in words:
        if word[::-1] in words and word[::-1] != word and (word[::-1], word) not in result:
            result.append((word, word[::-1]))
    return result

test_words = ["", "AB", "BA", "ABC", "EDF"]
for get_reverse_pairs in (get_reverse_pairs_naive, get_reverse_pairs_easy, get_reverse_pairs_sets):
    print(get_reverse_pairs)
    assert {("AB", "BA")} == set(get_reverse_pairs(test_words))

<function get_reverse_pairs_naive at 0x10ef749d8>
<function get_reverse_pairs_easy at 0x10f65cf28>
<function get_reverse_pairs_sets at 0x10f65cea0>


In [14]:
get_reverse_pairs_easy(cwords)

KeyboardInterrupt: 

## Exercise 3C

Modify the code of the previous exercise and read in all the words as keys of a dictionary (or a set) with arbitrary value (e.g. set the value to None) in each case. In this case use the in operator because bisect will only work on lists, and it will only work correctly, if the lists are sorted.

In [ ]:
def read_words_dict(path):
    with open(path) as f:
        return {line.strip(): None for line in f}
    
test_words_dict = {word: None for word in test_words}

## Same implementation as above, just better results because hashmap has fast lookup time

# Exercise 4

Anagrams

## Exercise 4A

Write a program that reads a word list from the file words.txt and prints all the sets of words that are anagrams.
Here is an example of what the output might look like:
          
          ['deltas', 'desalt', 'lasted', 'salted', 'slated', 'staled']
          ['retainers', 'ternaries']
          ['generating', 'greatening']
          ['resmelts', 'smelters', 'termless']

Hint: you might want to build a dictionary that maps from a set of letters to a list of words that can be spelled with those letters. The question is, how can you represent the set of letters in a way that can be used as a key?

In [15]:
## use disjoint union structure

from collections import Counter

def anagram_lists(words):
    hist = {word: Counter(word) for word in words}
    dju = [[word] for word in words]
    j = 0 
    while j < len(dju):
        moved = set()
        for i in range(j + 1, len(dju)):
            if hist[dju[i][0]] == hist[dju[j][0]]:
                dju[j].extend(dju[i])
                moved.add(i)   
        dju = [el for i, el in enumerate(dju) if i not in moved]
        j += 1

    return dju

test_words = ['deltas', 'desalt', 'lasted', 
              'salted', 'slated', 'staled', 
              'retainers', 'ternaries', 'generating', 
              'greatening', 'resmelts', 'smelters', 'termless', 
              'XXYZYZZZ', 'XYYZZXZZ', 'XYXYZZZZ', 'XYYZZZXZ', 'XZXYYZZZ', 'ZXXYYZZZ']

test_anagrams = [['deltas', 'desalt', 'lasted', 'salted', 'slated', 'staled'],
               ['retainers', 'ternaries'],
               ['generating', 'greatening'],
               ['resmelts', 'smelters', 'termless'],
               ['XXYZYZZZ', 'XYYZZXZZ', 'XYXYZZZZ', 'XYYZZZXZ', 'XZXYYZZZ', 'ZXXYYZZZ']]

assert test_anagrams == anagram_lists(test_words)

## Exercise 4B

Modify the previous program so that it prints the largest set of anagrams first, followed by the second largest set, and so on.

In [17]:
def sort_by_size(lol):
    """takes a list and sorts so the largest list comes first"""
    return sorted(lol, key=lambda l: len(l), reverse=True)

assert [] == sort_by_size([])
assert [[]] == sort_by_size([[]])
assert [[7,7,7,7,7,7], [4,4,4,4], [2,2,2], [1]] == sort_by_size([[1], [2,2,2], [7,7,7,7,7,7], [4,4,4,4]])
assert [[7,7,7,7,7,7], [4,4,4,4], [2,2,2], [1], []] == sort_by_size([[1], [2,2,2], [], [7,7,7,7,7,7], [4,4,4,4]])

## Exervise 4C

In Scrabble a “bingo” is when you play all seven tiles in your rack, along with a letter on the board, to form an eight-letter word. What set of 8 letters forms the most possible bingos? Hint: there are seven.

In [18]:
def find_bingos(words):
    als = sort_by_size(anagram_lists(words))
    als = [al for al in als if len(al[0]) == 8]
    return als

test_bingos = [['XXYZYZZZ', 'XYYZZXZZ', 'XYXYZZZZ', 'XYYZZZXZ', 'XZXYYZZZ', 'ZXXYYZZZ'],
               ['resmelts', 'smelters', 'termless']]

assert test_bingos == find_bingos(test_words)

In [ ]:
find_bingos(cwords)

# Exercise 5

Histograms

In [19]:
def histogram1(s):
    d = dict()
    for c in s:
        if c not in d:
            d[c] = 1 
        else:
            d[c] += 1
    return d
        
def histogram2(s):
    d = dict()
    for c in s:
        d[c] = d.get(c, 0) + 1
    return d

from collections import defaultdict

def histogram3(s):
    d = defaultdict(int)
    for c in s:
        d[c] += 1
    return d

def histogram4(s):
    return Counter(s)

for histogram in (histogram1, histogram2, histogram3, histogram4):
    print(histogram)
    assert histogram('') == {}
    assert histogram('a') == {'a': 1}
    assert histogram('abb') == {'a': 1, 'b': 2}

<function histogram1 at 0x10f65c950>
<function histogram2 at 0x10f652510>
<function histogram3 at 0x10f652378>
<function histogram4 at 0x10f652840>


## Exercise 5A

Take a look at the file ecoli-genome.fnafoundin\\bitsmb\groups\workshops\proglab2\ containing the DNA of a strain of escherichia coli. Write a function fasta_frequency(filename) that takes a filename as a parameter and returns a histogram containing the frequency of each nucleotide (or amino acid).

In [24]:
def parse_fasta_iterator(it):
    s = ''
    it = iter(it)
    for line in it:
        if not line.startswith('>'):
            s += line.strip()
    return s
        

test_fasta = """>gi|556503834|ref|NC_000913.3| Escherichia coli str. K-12 substr. MG1655, complete genome
AGCTTTTCATTCTGACTGCAACGGGCAATATGTCTCTGTGTGGATTAAAAAAAGAGTGTCTGATAGCAGC
"""
test_dna = "AGCTTTTCATTCTGACTGCAACGGGCAATATGTCTCTGTGTGGATTAAAAAAAGAGTGTCTGATAGCAGC"

assert test_dna == parse_fasta_iterator(test_fasta.split('\n'))

In [25]:
def open_fasta(path):
    with open(path) as f:
        return parse_fasta_iterator(f)

assert test_dna == open_fasta(mypath + 'test_fasta.txt')

def fasta_frequency(path):
    return Counter(open_fasta(path))

assert {'A': 20, 'C': 12, 'G': 17, 'T': 21} == fasta_frequency(mypath + 'test_fasta.txt')

## Exercise 5B

Write a function print_hist(h) that takes a histogram as parameter and prints a table of keys and values in alphabetical order of the keys.

In [26]:
def print_hist(h, file=sys.stdout):
    for key in sorted(h):
        print("{}: {}".format(key, h[key]), file=file)

test_output = io.StringIO()
print_hist(histogram("aaabbcd"), file=test_output)
assert 'a: 3\nb: 2\nc: 1\nd: 1\n' == test_output.getvalue()

## Exercise 5C

Combine the functions into a script where the user is asked to input a filename of a file in fasta format that calculates the frequencies and prints a table of nucleotides/amino acid symbols and their respective frequency.

In [27]:
def simulate_script_5c(argstring):
    parser = argparse.ArgumentParser()
    
    parser.add_argument("--input", "-i", default=sys.stdin, type=argparse.FileType('r'))
    parser.add_argument("--output", "-o", default=sys.stdout, type=argparse.FileType('w'))
    args = parser.parse_args(argstring.split())
    
    print_hist(histogram(parse_fasta_iterator(args.input)), file=args.output)
    
simulate_script_5c("--input /Users/cthoyt/GoogleDrive/Bonn/programming_lab_2/test_fasta.txt")

A: 20
C: 12
G: 17
T: 21


# Exercise 6

## Exercise 6A

Make sure that the function fasta_frequency(filename) from the previous exercise also works with multiple fasta sequences in one file. A single histogram should be returned combining the frequencies of all sequences. Calculate the histogram for the amino acids for all proteins combined.

## Exercise 6B

Now write a function that sorts the amino-acids by their relative frequency and print a table of amino acids and their relative frequency in decreasing order of frequency. Hint: The entries of a dictionary can be converted to a list of key–value tuples using the method items(). See section 12.6 of Think Python.

In [30]:
from operator import itemgetter

def sorted_histogram(d):
    return sorted(d.items(), key=itemgetter(1), reverse=True)

assert [('b', 6), ('a', 1), ('c', 0)] == sorted_histogram({'a':1, 'b':6, 'c':0})

## Exercise 6C

In order to make this more useful combine the functions in a single script fasta_frequency.py that asks the user for a filename and as a results prints the occurring amino-acids/nucleotides in decreasing order of frequency.

In [46]:
def simulate_script_6c(argstring):
    parser = argparse.ArgumentParser()
    
    parser.add_argument("--input", "-i", default=sys.stdin, type=argparse.FileType('r'))
    parser.add_argument("--output", "-o", default=sys.stdout, type=argparse.FileType('w'))
    parser.add_argument("--delimiter", '-d', default='\t')
    args = parser.parse_args(argstring.split())
    
    fasta = parse_fasta_iterator(args.input)
    hist = histogram(fasta)
    hist = sorted_histogram(hist)
    
    print(args.delimiter.join(("AA", "Frequency", "Percent")))
    total = sum(dict(hist).values())
    for k, v in hist:
        print(k, v, round(100 * v / total, 1), file=args.output, sep=args.delimiter)
    
simulate_script_6c("--input /Users/cthoyt/GoogleDrive/Bonn/programming_lab_2/test_fasta.txt")

AA	Frequency	Percent
T	21	30.0
A	20	28.6
G	17	24.3
C	12	17.1


## Exercise 6D

Frequently, scripts work in a non-interactive way. That is, they do not expect the user to enter anything while the program is running. Instead, all the information the program needs is passed to the program before the start using command line parameters. The program can then called from the command line:
         W:\> python fasta_frequency.py sequence.fasta
or from the notebook
         In [ ]: %run fasta_frequency.py sequence.fasta
Here, sequence.fasta is the filename containing the data you want to investigate. The parameters entered after the script name is available to the python script itself. It is stored in a list of strings that can be found in the module sys. E.g., consider a script named args.py
         import sys
         print(sys.argv) # argv is short for argument vector
Then running the script:
         In [ ]: %run args Was it a car or a cat I saw
will print
         ['args.py','Was','it','a','car','or','a','cat','I','saw']
Modify the script so that it reads the filename from the command line (to be found in sys.argv[1]).


### Solution

See use of argparse module above for more flexible user and developer options

## Exercise 6E

Apply your script to ecoli-proteome.faa and to one or more of the cromosomal proteomes of drosophila melangoster: drosophila-proteome-chrX.faa, drosophila-proteome-chr2.faa, drosophila-proteome-chr3.faa.
Are there noticeable differences in the amino acid composition between escherichia coli and drosophila melangoster?
How well do the frequencies correspond to the published results found in the publication amino-acid-composition.pdf (Fig. 1, p. 601):
5
Bogatyreva, N.; Finkelstein, A.V.; Galzitskaya, O.V. Trend of amino acid com- position of proteins of different taxa. Journal of Bioinformatics and Compu- tational Biology 4 (2006), 597–608.

In [47]:
simulate_script_6c("--input /Users/cthoyt/GoogleDrive/Bonn/programming_lab_2/drosophila-proteome-chrX.faa")

AA	Frequency	Percent
S	289627	8.9
L	278829	8.6
A	251815	7.8
G	220235	6.8
E	205421	6.3
T	190637	5.9
V	185844	5.7
Q	184048	5.7
P	182870	5.6
R	181199	5.6
D	166938	5.1
K	164383	5.1
N	153527	4.7
I	148514	4.6
F	102904	3.2
H	89491	2.8
Y	88730	2.7
M	74608	2.3
C	56592	1.7
W	29439	0.9
X	55	0.0
U	2	0.0


In [48]:
simulate_script_6c("--input /Users/cthoyt/GoogleDrive/Bonn/programming_lab_2/drosophila-proteome-chr2.faa")

AA	Frequency	Percent
L	302901	8.9
S	286645	8.4
A	253298	7.4
E	223008	6.6
G	208630	6.1
V	202001	5.9
T	194138	5.7
K	191612	5.6
P	189277	5.6
R	188988	5.6
D	179575	5.3
Q	177866	5.2
I	167175	4.9
N	162159	4.8
F	116587	3.4
Y	98769	2.9
H	88588	2.6
M	77553	2.3
C	58532	1.7
W	33124	1.0
X	78	0.0


In [49]:
simulate_script_6c("--input /Users/cthoyt/GoogleDrive/Bonn/programming_lab_2/drosophila-proteome-chr3.faa")

AA	Frequency	Percent
L	364026	8.8
S	358176	8.7
A	316168	7.7
G	260569	6.3
E	258114	6.3
V	238218	5.8
T	237067	5.8
P	232448	5.7
K	226375	5.5
Q	223395	5.4
R	222538	5.4
D	214280	5.2
N	195468	4.8
I	194081	4.7
F	139164	3.4
Y	115448	2.8
H	110799	2.7
M	97577	2.4
C	72069	1.8
W	37708	0.9
X	108	0.0


### Analysis

The values in these files were more dispuresd, with none coming high above 9%. There were also more amino acids with low frequencies than in the paper

# Exercise 7

A nice word puzzle and an even better programming exercise (Ex. 12.4 of Think Python) Here's another Car Talk Puzzler (http://www.cartalk.com/content/puzzlers):
What is the longest English word, that remains a valid English word, as you remove its letters one at a time?
Now, letters can be removed from either end, or the middle, but you can't rearrange any of the letters. Every time you drop a letter, you wind up with another English word. If you do that, you're eventually going to wind up with one letter and that too is going to be an English word—one that's found in the dictionary. I want to know what's the longest word and how many letters does it have?

I'm going to give you a little modest example: Sprite. Ok? You start off with sprite, you take a letter off, one from the interior of the word, take the r away, and we're left with the word spite, then we take the e off the end, we're left with spit, we take the s off, we're left with pit, it, and I.
Write a program to find all words that can be reduced in this way, and then find the longest one.
This exercise is a little more challenging than most and a few hints are given for solving this problem are given in Think Python. When trying this exercise please note that the provided list of words words.txt does not contain the one letter words ‘a' and ‘i'.


### Solution

- Construct a graph where each node represents a word. 
- For every pair of nodes i and j, if i can be made by taking a letter off of j, add a directed edge (i, j)
- Find all paths from 1 letter words. the terminal nodes of these paths are all solutions

In [7]:
def match_minus(a, b):
    """returns true if a can be constructed by removing a letter from b"""
    if len(a) + 1 != len(b):
        return False
    for i in range(len(b)):
        if a[:i] == b[:i] and a[i:] == b[i+1:]:
            return True
    return False

assert match_minus('spite', 'sprite')
assert not match_minus('sprite', 'spite')
assert not match_minus('spit', 'sprite')
assert match_minus('i', 'ih')
assert match_minus('i', 'hi')
assert match_minus('', 'i')

In [58]:
from collections import defaultdict

def build_graph(words):
    g = defaultdict(set)
    mwords = list(words) + ['a', 'i']
    
    ltw = defaultdict(list)
    for word in words:
        ltw[len(word)].append(word)
    
    for l in sorted(ltw.keys()):
        for a, b in product(ltw[l], ltw[l + 1]):
            if match_minus(a, b):
                g[a].add(b)
    
    return dict(g)

test_words = ['i', 'hi', 'his', 'him', 'in', 'shim']

assert {'i': {'in', 'hi'}, 'him': {'shim'}, 'hi': {'him', 'his'}} == build_graph(test_words)

def all_paths(g, start_node):
    if start_node not in g:
        return [[start_node]]
    paths = []
    for neighbor in g[start_node]:
        paths.extend(all_paths(g, neighbor))
    return [[start_node] + path for path in paths]

assert sorted([['i', 'in'], 
               ['i', 'hi', 'him', 'shim'], 
               ['i', 'hi', 'his']]) == sorted(all_paths(build_graph(test_words), 'i'))

def get_solutions(words):
    g = build_graph(words)
    print('graph built')
    paths = all_paths(g, 'i') + all_paths(g, 'a')
    solutions = set()
    for path in paths:
        for el in path:  
            solutions.add(el)
    return solutions
    
assert {'shim', 'his', 'him', 'in', 'hi', 'i', 'a'} == get_solutions(test_words)
assert 'shim' == max(get_solutions(test_words), key=len)

graph built


In [ ]:
get_solutions(cwords)

In [17]:
from collections import defaultdict
from itertools import product

with open('/Users/cthoyt/GoogleDrive/Bonn/programming_lab_2/words.txt') as f:
    words = [line.strip() for line in f]
words[:10]

len2words = defaultdict(list)

for word in words:
    len2words[len(word)].append(word)
    
for l in len2words:
    len2words[l] = sorted(len2words[l])
    print("length={} number={}".format(l, len(len2words[l])))

lengths = sorted(len2words)
print(lengths)
last = len2words[lengths[0]]
g = defaultdict(set)
r = defaultdict(set)

for l in lengths[1:]:
    new_last = set()
    print("l:{} length of l-1:{}, length of l:{}".format(l, len(last), len(len2words[l])))
    for a, b in product(last, len2words[l]):
        if match_minus(a, b):
            g[a].add(b)
            r[b].add(a)
            new_last.add(b)
    last = new_last

length=2 number=85
length=3 number=908
length=4 number=3686
length=5 number=8258
length=6 number=14374
length=7 number=21727
length=8 number=26447
length=9 number=16658
length=10 number=9199
length=11 number=5296
length=12 number=3166
length=13 number=1960
length=14 number=1023
length=15 number=557
length=16 number=261
length=17 number=132
length=18 number=48
length=19 number=16
length=20 number=5
length=21 number=3
[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21]
l:3 length of l-1:85, length of l:908
l:4 length of l-1:681, length of l:3686
l:5 length of l-1:2489, length of l:8258
l:6 length of l-1:4355, length of l:14374
l:7 length of l-1:4338, length of l:21727
l:8 length of l-1:2867, length of l:26447
l:9 length of l-1:1163, length of l:16658
l:10 length of l-1:210, length of l:9199
l:11 length of l-1:9, length of l:5296
l:12 length of l-1:1, length of l:3166
l:13 length of l-1:0, length of l:1960
l:14 length of l-1:0, length of l:1023
l:15 length of l-1:0, l

In [52]:
for word in len2words[2]:
    if match_minus('i', word):
        g['i'].add(word)
    if match_minus('a', word):
        g['a'].add(word)

In [53]:
# do a depth first seach on g

res = set()

def dfs(g, s):
    if s not in g:
        res.add(s)
    elif 0 == len(g[s]):
        res.add(s)
    else:
        for neighbor in g[s]:
            dfs(g, neighbor)
            
dfs(g, 'i')
dfs(g, 'a')


In [54]:
res = sorted(res, key=len, reverse=True)
print(len(res), res[0:2])

3496 ['complecting', 'restarting']
